# BAT.jl Example

The [the tutorial section](https://bat.github.io/BAT.jl/stable/tutorial/) of the BAT.jl documentation should prove helpful.

The cell below installs all the neccessary packages for the example including BAT.

In [ ]:
using Pkg
Pkg.add("BAT")
Pkg.add("Distributions")
Pkg.add("IntervalSets")
Pkg.add("ValueShapes")
Pkg.add("Plots")
Pkg.add("ArraysOfArrays")
Pkg.add("StatsBase")
Pkg.add("LinearAlgebra")
Pkg.add("Measurements")

In [ ]:
using BAT
using Distributions 
using IntervalSets
using ValueShapes
using Plots
using ArraysOfArrays
using StatsBase 
using LinearAlgebra
using Measurements
pyplot()

## Situation - Poisson Counting Example
We want to determine the properties of a radioactive singal source in the presence of background from natural sources of radioactivity.
We assume to have one signal source $S$ and only one source of background $B$.

## 1. Background only measurement
+ $k_B=10$ counts with detector without source
+ Determine event rate of natural radioactive background

We define the log-likelihood function, using the function `logpdf()` and type `Poisson` provided by the package [Distributions.jl](https://juliastats.github.io/Distributions.jl/latest/univariate/):

In [ ]:
# Number of observed background events
kb = 10

likelihood_B = let k = kb
    params -> begin
        return logpdf(Poisson(params.λb), k) # poisson log-likelihood
    end
end;

Define the Prior with help of the `NamedTupleDist()` function. Use a flat prior between 0 and 30:

Then use the likelihood and the prior to define the `PosteriorDensity()`:

In [ ]:
prior_B = NamedTupleDist(
    λb = 0..30 
);
posterior_B = PosteriorDensity(likelihood_B, prior_B);

Define the settings for the sampling. Choose `MetropolisHastings()` as the algorithm and set the number of chains and samples:

In [ ]:
algorithm = MetropolisHastings()
nchains = 8
nsamples = 10^5;

Use the function `bat_sample` to sample the posterior using `nchains` MCMC chains, generating `nsamples` per chain:

In [ ]:
samples_B, stats_B = bat_sample(posterior_B, (nsamples, nchains), algorithm)
stats_B

Take a look at the resulting disribution for the background rate using `plot()`:

In [ ]:
plot(posterior_B, samples_B, :λb)
plot!(prior_B, :λb, xlabel = "\$\\lambda_b\$", ylabel = "\$P(\\lambda_b)\$")

Print some statistics of the samples:

In [ ]:
println("Mode: $(stats_B.mode)")
println("Mean: $(stats_B.mean)")
println("Covariance: $(stats_B.cov)")
println("Standard Deviation: $(diag(sqrt(stats_B.cov[:, :])))")

## 2. Second background only measurement
+ Second background measurement with $k_B=8$ counts.  
+ Update estimation of background rate 
+ Similar procedure
+ Use the posterior distribution of the previous background measurement as a prior


Define a `StatsBase` histogram containing the previous posterior distribution.

Define it as the prior using `BAT.HistogramAsUvDistribution(histogram)`.  

In [ ]:
posterior_hist_B1 = fit(Histogram, flatview(samples_B.params)[1, :], FrequencyWeights(samples_B.weight), nbins = 400);

prior_B2 = NamedTupleDist(
    λb = BAT.HistogramAsUvDistribution(posterior_hist_B1)
)

Rest is analogous to the first example.

Define the log-likelihood function and the posterior.

In [ ]:
kb2 = 8

likelihood_B2 = let k = kb2
    params -> logpdf(Poisson(params.λb), k)
end;

posterior_B2 = PosteriorDensity(likelihood_B2, prior_B2);

Generate samples.

In [ ]:
samples_B2, stats_B2 = bat_sample(posterior_B2, (nsamples, nchains), algorithm)
stats_B2

Use the  `plot(samples)` and `plot!(prior)` functions to visualize the posterior of the first analysis and the updated posterior together:

In [ ]:
plot(posterior_B2, samples_B2, :λb)
plot!(prior_B2, :λb, linewidth=1.5, xlabel = "\$\\lambda_b\$", ylabel = "\$P(\\lambda_b)\$")

Print some statistics of the samples:

In [ ]:
println("Mode: $(stats_B2.mode)")
println("Mean: $(stats_B2.mean)")
println("Covariance: $(stats_B2.cov)")
println("Standard Deviation: $(diag(sqrt(stats_B2.cov[:, :])))")

## 3. Signal + Background
+ $k_{S+B}=12$ counts using the radioactive source in the setup
+ Determine signal rate $\lambda_s$ using knowledge about background

Define the likelihood for the signal + background model.

In [ ]:
# Number of observed events
kSB = 12

likelihood_SB = let k = kSB
    params -> begin
        return logpdf(Poisson(params.λb + params.λs), k)  # poisson log-likelihood for b+s
    end
end;

Define the prior for both the signal and backgound parameters.  
Signal prior is chosen flat while background prior uses the distribution from the earlier measurement.

In [ ]:
hist_B2 = fit(Histogram, flatview(samples_B2.params)[1, :], FrequencyWeights(samples_B2.weight), nbins = 400)

B2 = BAT.HistogramAsUvDistribution(hist_B2);

prior_SB = NamedTupleDist(
    λb = B2,
    λs = 0..30
)

posterior_SB = PosteriorDensity(likelihood_SB, prior_SB);

Generate samples for the signal + background model:

In [ ]:
samples_SB, stats_SB = bat_sample(posterior_SB, (nsamples, nchains), algorithm)
stats_SB

Plot an overview of the results for both prameters using `plot(samples)`.   

In [ ]:
plot(samples_SB, param_labels=["\\lambda_b","\\lambda_s"])

Print some statistics of the samples:

In [ ]:
println("Mode: $(stats_SB.mode)")
println("Mean: $(stats_SB.mean)")
println("Covariance: $(stats_SB.cov)")
println("Standard Deviation: $(diag(sqrt(stats_SB.cov[:, :])))")

## 4. Error propagation

+ Calculate cross section using 
### $\frac{\mathrm d N}{\mathrm d t} = \epsilon \cdot σ \cdot L$ 
with the Luminosity $L$ and the efficiency of the detector $\epsilon$. 
+ Can be rewritten to 
### $σ_S = \frac{λ_s}{\epsilon \cdot L}$  .
+ For this axample we assume a luminosity $L = 1.1$  (neglecting units).
+ Result either a measurement or an upper limit on the signal cross section.

+ Assume a known efficiency of $\epsilon = 0.1 \pm 0.02$

Define the luminosity and the efficiency:

In [ ]:
L = 1.1
ϵ = rand(Normal(0.1, 0.02), nsamples);

Plot the efficiency.  

In [ ]:
hist_ϵ = fit(Histogram, ϵ, nbins=200)
plot(hist_ϵ, 1, seriestype=:smallest_intervals, xlabel="\$\\epsilon\$", ylabel="\$p(\\epsilon)\$")

Get unweighted samples for the signal rate and calculate the cross section distribution:

In [ ]:
resampled_SB = bat_sample(samples_SB, nsamples).samples;
λ_SB = valshape(prior_SB).(resampled_SB.params).λs
σS = (λ_SB)./(ϵ*L);

Comparing the error propation using the `Measurements` package to the sample

In [ ]:
λ_SB_M = measurement(mean(λ_SB), sqrt(var(λ_SB)))
ϵ_M = measurement(0.1, 0.02)
println(string("Using gaussian error propation: ",λ_SB_M/(ϵ_M*L)))

σS_M = measurement(mean(σS), sqrt(var(σS)))
println(string("Propagating uncertainty with sampling: ",σS_M))

Using sampling we can plot the distribution of the signal cross section.

In [ ]:
hist_σ = fit(Histogram, σS, nbins=300)
plot(hist_σ, 1, seriestype=:smallest_intervals, xlim=(0,400), xlabel="\$\\sigma_s\$", ylabel="\$p(\\sigma_s)\$")